## Machine learning Models and methods followed

- Problem statement:-
- Extract 3000 reviews for Lionking movie which was released in july 2019 from rotten tomatoes.com
- Extract these columns:
- a. ReviewID
- b. Reviewer Name
- c. Review
- d. Rating
- e. Date-of-Review
- Label the Review Sentiment(0 for "positive" and 1 for "negative) based on rating >3 'Positive' else 'Negtive binary class
- Need to develop visualizations to find any rellationship between the features to the Sentiment whether good/bad from the data

- This particular assessment of reviews helps customers before going a movie,or purchasing. 
- For production house they will know which kind of movies the audience will like how they are percieving the movie.

- For online sites they would know how many users are accessing their portal  and to grow their business they can  recommend similarly liked other movies to the users.




- Methods followed
- Initially focused on preprocessing steps i tried with 8 preprocessing steps along with stopwords removal lemmetization included Common words , and rare words,punctuations,white spaces,numbers removal but my F1score did n't go beyond 50% with TFIDF vectorizer.
- Then i tried parameter tuning in TFIDF vectorizer with min_df,max_df by which my accuracy score started to increase.
- as the reviews dataset is class imbalanced dataset addressed This problem by assigning  class_weights to 'balanced'.
- Naive Bayes classifier F1 score max:-0.5610200364298725
- Logistic Regression  F1 Score   max:- : 0.6523955147808359
-  Random Forest Classifier using Gridsearch best parameters F1 score max : 0.7006
-  XGBoost classifier using Gridsearch best parameters         F1 score max : 0.8188

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
data=pd.read_csv("lionking_Sentiment_reviews.csv")
data.head()

,Unnamed: 0,user,userid,review_text,createDate,updateDate,rating,Sentiment
0,0,Heather,21FFC7F2-E843-4473-A525-DE7AE336C459,8 year old loved it. stayed true cartoon book....,2019-08-24,2019-08-24,5.0,0
1,1,Kat R,977265260,"I love every minute it. Pumba hilarious, Timon...",2019-08-24,2019-08-24,5.0,0
2,2,Culs,43058975-15d6-4d5a-9565-28d67fd831c8,Theres nothing exciting I saw. The voices fit ...,2019-08-24,2019-08-24,2.0,1
3,3,Steven L,785369549,The live action adaptation great. Loved movie....,2019-08-24,2019-08-24,4.5,0
4,4,Wendy C,9fc3e02a-b2a2-435c-b3ba-f827a96e9e64,I enjoyed movie. I little disappointed Timon P...,2019-08-24,2019-08-24,4.0,0


In [3]:
data["ReviewID"]=data["userid"]

In [4]:
data.drop(["Unnamed: 0","userid"],axis=1,inplace=True)

In [5]:
data.head(3)

,user,review_text,createDate,updateDate,rating,Sentiment,ReviewID
0,Heather,8 year old loved it. stayed true cartoon book....,2019-08-24,2019-08-24,5.0,0,21FFC7F2-E843-4473-A525-DE7AE336C459
1,Kat R,"I love every minute it. Pumba hilarious, Timon...",2019-08-24,2019-08-24,5.0,0,977265260
2,Culs,Theres nothing exciting I saw. The voices fit ...,2019-08-24,2019-08-24,2.0,1,43058975-15d6-4d5a-9565-28d67fd831c8


## creating number of words count in review_text 

In [6]:
data["num_words"] = data['review_text'].str.split().str.len() #Calculating the number of words in the reviewtext

In [7]:
data.head()

,user,review_text,createDate,updateDate,rating,Sentiment,ReviewID,num_words
0,Heather,8 year old loved it. stayed true cartoon book....,2019-08-24,2019-08-24,5.0,0,21FFC7F2-E843-4473-A525-DE7AE336C459,10
1,Kat R,"I love every minute it. Pumba hilarious, Timon...",2019-08-24,2019-08-24,5.0,0,977265260,17
2,Culs,Theres nothing exciting I saw. The voices fit ...,2019-08-24,2019-08-24,2.0,1,43058975-15d6-4d5a-9565-28d67fd831c8,9
3,Steven L,The live action adaptation great. Loved movie....,2019-08-24,2019-08-24,4.5,0,785369549,35
4,Wendy C,I enjoyed movie. I little disappointed Timon P...,2019-08-24,2019-08-24,4.0,0,9fc3e02a-b2a2-435c-b3ba-f827a96e9e64,24


In [8]:
import numpy as np
import pandas as pd
import os
from textblob import TextBlob #library for processing textual data
import re #regular expression re Identifiers, and White space characters

## 1.transform our reviewtext into lower case.This avoids having multiple copies of the same words¶

In [9]:
data['review_text'] = data['review_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['review_text'].head()

0    8 year old loved it. stayed true cartoon book....
1    i love every minute it. pumba hilarious, timon...
2    theres nothing exciting i saw. the voices fit ...
3    the live action adaptation great. loved movie....
4    i enjoyed movie. i little disappointed timon p...
Name: review_text, dtype: object

## 2.Removing Punctuation

In [10]:
data['review_text'] = data['review_text'].str.replace('[^\w\s]','')

## 3.lemmetize

In [11]:
from textblob import Word
data['review_text'] = data['review_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['review_text'].head()

0    8 year old loved it stayed true cartoon book l...
1    i love every minute it pumba hilarious timon i...
2    there nothing exciting i saw the voice fit cha...
3    the live action adaptation great loved movie e...
4    i enjoyed movie i little disappointed timon pu...
Name: review_text, dtype: object

##  4spelling correction

In [12]:
from textblob import TextBlob
data['review_text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    8 year old loved it stayed true cartoon book l...
1    i love every minute it gumma hilarious simon i...
2    there nothing exciting i saw the voice fit cha...
3    the live action adaptation great loved movie e...
4    i enjoyed movie i little disappointed simon gu...
Name: review_text, dtype: object

In [13]:
## 5.Stop words removal

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['review_text'] = data['review_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['review_text'].head()

0      8 year old loved stayed true cartoon book loved
1    love every minute pumba hilarious timon loved ...
2             nothing exciting saw voice fit character
3    live action adaptation great loved movie even ...
4    enjoyed movie little disappointed timon pumba ...
Name: review_text, dtype: object

## 5.Removing white spaces and numbers 

In [15]:
##Removing white spaces and numbers 
data['review_text'] = data['review_text'].apply(lambda x:re.sub(r'\s+', ' ', x))

In [16]:
#removing Numbers
data['review_text'] = data['review_text'].apply(lambda x:re.sub('[\d]', '',x))


In [17]:
data['review_text_preprocessed']=data['review_text']

In [18]:
data.head()

,user,review_text,createDate,updateDate,rating,Sentiment,ReviewID,num_words,review_text_preprocessed
0,Heather,year old loved stayed true cartoon book loved,2019-08-24,2019-08-24,5.0,0,21FFC7F2-E843-4473-A525-DE7AE336C459,10,year old loved stayed true cartoon book loved
1,Kat R,love every minute pumba hilarious timon loved ...,2019-08-24,2019-08-24,5.0,0,977265260,17,love every minute pumba hilarious timon loved ...
2,Culs,nothing exciting saw voice fit character,2019-08-24,2019-08-24,2.0,1,43058975-15d6-4d5a-9565-28d67fd831c8,9,nothing exciting saw voice fit character
3,Steven L,live action adaptation great loved movie even ...,2019-08-24,2019-08-24,4.5,0,785369549,35,live action adaptation great loved movie even ...
4,Wendy C,enjoyed movie little disappointed timon pumba ...,2019-08-24,2019-08-24,4.0,0,9fc3e02a-b2a2-435c-b3ba-f827a96e9e64,24,enjoyed movie little disappointed timon pumba ...


## creating number of words count after preprocessed to check 

In [19]:
data["num_words_preprocessed "] = data['review_text_preprocessed'].str.split().str.len() #Calculating the number of words in the reviewtext

In [20]:
X_train, X_test, Y_train, Y_test=train_test_split(data,data['Sentiment'],test_size=0.3,random_state=5555)

In [21]:
X_train.head(3)

,user,review_text,createDate,updateDate,rating,Sentiment,ReviewID,num_words,review_text_preprocessed,num_words_preprocessed
426,Kevin M,good go mainly sentimental value,2019-08-18,2019-08-18,4.0,0,21FE7D93-351C-43B5-833D-B1DB11F3FD6A,5,good go mainly sentimental value,5
2725,Handpatter,think ridiculous would lion king preview lion ...,2019-08-04,2019-08-04,5.0,0,4849406C-3146-46C3-A4AF-8B6CE44BE07D,43,think ridiculous would lion king preview lion ...,36
1963,Keeks,movie good great believe duet donald glover be...,2019-08-06,2019-08-06,4.0,0,aa85dc28-de08-41bc-82fb-91957507c6d4,33,movie good great believe duet donald glover be...,29


In [22]:
Y_train.value_counts()

0    1731
1     369
Name: Sentiment, dtype: int64

In [23]:
data.head()

,user,review_text,createDate,updateDate,rating,Sentiment,ReviewID,num_words,review_text_preprocessed,num_words_preprocessed
0,Heather,year old loved stayed true cartoon book loved,2019-08-24,2019-08-24,5.0,0,21FFC7F2-E843-4473-A525-DE7AE336C459,10,year old loved stayed true cartoon book loved,8
1,Kat R,love every minute pumba hilarious timon loved ...,2019-08-24,2019-08-24,5.0,0,977265260,17,love every minute pumba hilarious timon loved ...,13
2,Culs,nothing exciting saw voice fit character,2019-08-24,2019-08-24,2.0,1,43058975-15d6-4d5a-9565-28d67fd831c8,9,nothing exciting saw voice fit character,6
3,Steven L,live action adaptation great loved movie even ...,2019-08-24,2019-08-24,4.5,0,785369549,35,live action adaptation great loved movie even ...,30
4,Wendy C,enjoyed movie little disappointed timon pumba ...,2019-08-24,2019-08-24,4.0,0,9fc3e02a-b2a2-435c-b3ba-f827a96e9e64,24,enjoyed movie little disappointed timon pumba ...,19


In [24]:
data.drop(["user","updateDate"],axis=1,inplace=True)

In [25]:
data.head()

,review_text,createDate,rating,Sentiment,ReviewID,num_words,review_text_preprocessed,num_words_preprocessed
0,year old loved stayed true cartoon book loved,2019-08-24,5.0,0,21FFC7F2-E843-4473-A525-DE7AE336C459,10,year old loved stayed true cartoon book loved,8
1,love every minute pumba hilarious timon loved ...,2019-08-24,5.0,0,977265260,17,love every minute pumba hilarious timon loved ...,13
2,nothing exciting saw voice fit character,2019-08-24,2.0,1,43058975-15d6-4d5a-9565-28d67fd831c8,9,nothing exciting saw voice fit character,6
3,live action adaptation great loved movie even ...,2019-08-24,4.5,0,785369549,35,live action adaptation great loved movie even ...,30
4,enjoyed movie little disappointed timon pumba ...,2019-08-24,4.0,0,9fc3e02a-b2a2-435c-b3ba-f827a96e9e64,24,enjoyed movie little disappointed timon pumba ...,19


In [26]:
## splitting data into train test

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(data,data['Sentiment'],test_size = 0.30, random_state = 12321)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

## construct TFIDF matrix on train documents 

In [29]:
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1),stop_words='english',max_df = 1.00, min_df = 0.005)
X_train_tfidf = pd.DataFrame(tfidf_transformer.fit_transform(X_train['review_text_preprocessed']).todense(), 
                             columns = tfidf_transformer.get_feature_names())
print(X_train_tfidf.shape)
# Get the tfidf matrix for test documents
X_test_tfidf = pd.DataFrame(tfidf_transformer.transform(X_test['review_text_preprocessed']).todense(),
                            columns = tfidf_transformer.get_feature_names())
print(X_test_tfidf.shape)

(2100, 359)
(900, 359)


- max_df = 1.00 means ignore terms that appear in more than 100% of the documents
- min_df = 0.05 means ignore terms that appear in less than 5% of the documents

### Implementing Naive Bayes Classification model

In [30]:
from sklearn.naive_bayes import MultinomialNB
nb_clf=MultinomialNB().fit(X_train_tfidf, Y_train) #nb_clf is the classifier
pred_train = nb_clf.predict(X_train_tfidf) #get predictions on train data


In [31]:
from sklearn.metrics import f1_score

print("F1 Score: {}".format(f1_score(Y_train,pred_train)))

F1 Score: 0.4729458917835671


In [32]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(class_weight='balanced', random_state=12321)

In [33]:
logreg = linear_model.LogisticRegression(class_weight='balanced', random_state=12321)
lr_clf = logreg.fit(X_train_tfidf, Y_train) #fit logreg model on train data

## Implementing Logistic Regression

In [34]:
from sklearn import linear_model
logreg_train = linear_model.LogisticRegression(class_weight='balanced', random_state=55555)
lr_clf = logreg_train.fit(X_train_tfidf, Y_train) #fit logreg model on train data

pred_lr_train=lr_clf.predict(X_train_tfidf) # predict on train data
pred_lr_test=lr_clf.predict(X_test_tfidf) # predict on test data
print("F1 Score: {}".format(f1_score(Y_train,pred_lr_train)))

F1 Score: 0.6523955147808359


-logistic regression F1 score is 6523955147808359

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model

In [36]:
logistic = linear_model.LogisticRegression()

In [37]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)

In [38]:
log_clf_grid = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

In [39]:
best_model = log_clf_grid.fit(X_train_tfidf, Y_train)

In [40]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l2
Best C: 2.7825594022071245


In [41]:
logistic_final = linear_model.LogisticRegression(penalty="l2", C = 2.782559402207124)

In [42]:
log_clf_grid.fit(X_train_tfidf, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.00000e+00, 2.78256e+00, 7.74264e+00, 2.15443e+01, 5.99484e+01,
       1.66810e+02, 4.64159e+02, 1.29155e+03, 3.59381e+03, 1.00000e+04]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
pred_final_train=log_clf_grid.predict(X_train_tfidf) # predict on train data

In [44]:
red_final_test=log_clf_grid.predict(X_test_tfidf)#predict on test data

In [45]:
print("F1 Score: {}".format(f1_score(Y_train,pred_final_train)))

F1 Score: 0.6262975778546713


In [46]:
from xgboost import XGBClassifier
XGB_model = XGBClassifier()

In [47]:
XGB_model_train= XGB_model.fit(X=X_train_tfidf,y=Y_train)

In [48]:
pred_train_XG = XGB_model.predict(X_train_tfidf)
pred_test_XG = XGB_model.predict(X_test_tfidf)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [49]:
print("F1 Score: {}".format(f1_score(Y_train,pred_train_XG)))

F1 Score: 0.5030425963488844


In [50]:
print("F1 Score: {}".format(f1_score(Y_test,pred_test_XG)))

F1 Score: 0.3251231527093596


In [51]:
xgb_grid = XGBClassifier(random_state=123)
param_grid = {
    'learning_rate': [0.2,0.4,0.6,0.8],
    'n_estimators': [50,60,70,80],
    'reg_alpha': [0.4,0.6],
    'colsample_bytree': np.linspace(0.5,0.9,3)    
    }

xgb_grid = GridSearchCV(estimator = xgb_grid, param_grid = param_grid, cv = 3)
best_model = xgb_grid.fit(X_train_tfidf, Y_train)

pred_xgb_train=best_model.predict(X_train_tfidf)
pred_xgb_test=best_model.predict(X_test_tfidf)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

...

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future thi

In [52]:
print(classification_report(Y_train, pred_xgb_train  ,digits=4))

             precision    recall  f1-score   support

          0     0.8931    0.9977    0.9425      1733
          1     0.9756    0.4360    0.6026       367

avg / total     0.9075    0.8995    0.8831      2100



- F1score on train is 0.8966 


In [53]:
print(classification_report(Y_test,pred_xgb_test,digits=4))

             precision    recall  f1-score   support

          0     0.8550    0.9851    0.9154       736
          1     0.7885    0.2500    0.3796       164

avg / total     0.8428    0.8511    0.8178       900



-XGboost F1 score is  0.8188

## Random forest 

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

- using gridsearch to tune theparameter 

In [56]:
rfc_grid = RandomForestClassifier(n_jobs=-1, max_features='sqrt',class_weight='balanced',oob_score=True)
param_grid = { 
           "n_estimators" :[30,50,60,70,80,100,200],
           "max_depth" : [10,20,30],
           "min_samples_leaf" : [50,100]}
 
rfc_cv_grid = GridSearchCV(estimator = rfc_grid, param_grid = param_grid, cv = 10)

- n_jobs = -1   it uses all cores of processor
- max_features is the maximum number of attributes to select for each tree
 
- n_estimators is the number of trees in the forest

- max_depth is how deep each tree can be
-min_sample_leaf is the minimum samples required in each leaf node for the root node to split
- "A node will only be split if in each of it's leaf nodes there should be min_sample_leaf"


In [57]:
rfc_cv_grid.fit(X = X_train_tfidf, y = Y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [30, 50, 60, 70, 80, 100, 200], 'max_depth': [10, 20, 30], 'min_samples_leaf': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [59]:
print(rfc_cv_grid.best_score_,rfc_cv_grid.best_params_)

0.6823809523809524 {'max_depth': 30, 'min_samples_leaf': 50, 'n_estimators': 200}


In [60]:
pred_rfc_train=rfc_cv_grid.predict(X_train_tfidf)

In [61]:
pred_rfc_test=rfc_cv_grid.predict(X_test_tfidf)

In [62]:
print(classification_report(Y_train, pred_rfc_train ,digits=4))

             precision    recall  f1-score   support

          0     0.9443    0.6746    0.7869      1733
          1     0.3457    0.8120    0.4849       367

avg / total     0.8397    0.6986    0.7342      2100



In [63]:
print(classification_report(Y_test,pred_rfc_test,digits=4))

             precision    recall  f1-score   support

          0     0.9261    0.6807    0.7847       736
          1     0.3454    0.7561    0.4742       164

avg / total     0.8203    0.6944    0.7281       900



- RandomForest F1 score is  0.7281

In [64]:
from sklearn.svm import SVC
svc_grid = SVC(random_state=555)

In [65]:
param_grid = {

'C': [0.001, 0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1], 
'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
svc_cv_grid = GridSearchCV(estimator = svc_grid, param_grid = param_grid, cv = 5)

In [66]:
svc_cv_grid.fit(X = X_train_tfidf, y = Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=555, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1, 1], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [68]:
print(svc_cv_grid.best_score_,svc_cv_grid.best_params_)

0.8604761904761905 {'C': 1, 'gamma': 1, 'kernel': 'sigmoid'}


In [69]:
svc_c1_sigmoid = SVC(C=1,gamma=1,kernel='sigmoid',random_state=5555)

In [70]:
svc_c1_sigmoid.fit(X = X_train_tfidf, y = Y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=5555, shrinking=True,
  tol=0.001, verbose=False)

In [71]:
pred_svc_train_cv = svc_c1_sigmoid.predict(X_train_tfidf)

In [72]:
pred_svc_test_cv = svc_c1_sigmoid.predict(X_test_tfidf)

In [73]:
print("F1 Score: {}".format(f1_score(Y_train,pred_svc_train_cv)))

F1 Score: 0.49812734082397


In [74]:
print("F1 Score: {}".format(f1_score(Y_test,pred_svc_test_cv)))

F1 Score: 0.3930131004366812


-F1 score SVC is 0.3930131004366812

# predict on test data 

In [75]:
data1=pd.read_csv("test-1566381431512.csv")

In [76]:
data1.head()

,ReviewID,Review
0,92876,Was good. Nothing like the original but I beli...
1,92877,I absolutely loved it! A wonderful rendition o...
2,92878,I love the movie! Good job director! \nI appre...
3,92879,"GREAT MOVIE!! Same as the original, but the c..."
4,92880,Realistic. Fantastic special effects


## transform our reviewtext into lower case.This avoids having multiple copies of the same words¶

In [77]:
data1['Review'] = data1['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data1['Review'].head()

0    was good. nothing like the original but i beli...
1    i absolutely loved it! a wonderful rendition o...
2    i love the movie! good job director! i appreci...
3    great movie!! same as the original, but the ci...
4                 realistic. fantastic special effects
Name: Review, dtype: object

## 2.removing punctuations 

In [78]:
data1['Review'] = data1['Review'].str.replace('[^\w\s]','')

## 3.lemmetize

In [79]:
from textblob import Word
data1['Review'] = data1['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data1['Review'].head()

0    wa good nothing like the original but i believ...
1    i absolutely loved it a wonderful rendition of...
2    i love the movie good job director i appreciat...
3    great movie same a the original but the cinema...
4                   realistic fantastic special effect
Name: Review, dtype: object

In [82]:
#Removing white spaces 
data1['Review'] = data1['Review'].apply(lambda x:re.sub(r'\s+', ' ', x))

In [83]:
#removing Numbers
data1['Review'] = data1['Review'].apply(lambda x:re.sub('[\d]', '',x))


## 4spelling correction

In [85]:
from textblob import TextBlob
data1['Review'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    wa good nothing like the original but i believ...
1    i absolutely loved it a wonderful condition of...
2    i love the movie good job director i appreciat...
3    great movie same a the original but the cinema...
4                   realistic fantastic special effect
Name: Review, dtype: object

In [86]:
data1_tfidf = pd.DataFrame(tfidf_transformer.transform(data1['Review']).todense(),
                            columns = tfidf_transformer.get_feature_names())


In [87]:
data1_tfidf.shape

(1200, 359)

In [101]:
import warnings
warnings.filterwarnings("ignore")

## xgb gridsearch 

In [103]:
pred_xgb_grid=best_model.predict(data1_tfidf)

In [104]:
data1["sentiment"]=pred_xgb_grid

In [105]:
data1[['ReviewID','sentiment']].to_csv("pred_xgb_grid_again2.csv",index = False)

## Random forest gridsearch 

In [106]:
pred_rfc_grid_test=rfc_cv_grid.predict(data1_tfidf)

In [107]:
data1["sentiment"]=pred_rfc_grid_test

In [108]:
data1[['ReviewID','sentiment']].to_csv("RFC_grid_cv_again.csv",index = False)

In [109]:
pred_rfc_cv_grid = rfc_cv_grid.predict(data1_tfidf)

In [110]:
pred_logisitc_clf=lr_clf.predict(data1_tfidf)

In [111]:
data1["Sentiment"]=pred_logisitc_clf

In [99]:
data1.head(2)

,ReviewID,Review,sentiment,Sentiment
0,92876,wa good nothing like the original but i believ...,1,0
1,92877,i absolutely loved it a wonderful rendition of...,0,0


In [100]:
data1[['ReviewID','Sentiment']].to_csv("logistic_class_weight_data1_numb_spaces.csv",index = False)